In [15]:
import pandas as pd
from datetime import datetime, timedelta
import os
import pandas as pd
from pathlib import Path
import os
import datetime
import holidays
import numpy as np

# Function to generate a DataFrame with missing rows added
def fill_missing_values(file_path, sheet_name=None):
    # Read the Excel file
    if sheet_name:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=4)  # Set the fifth row as the header
    else:
        df = pd.read_excel(file_path, header=4)  # Set the fifth row as the header

    # Print column names to verify
    print("Columns:", df.columns)

    # Access 'Date' column by index (assumed to be the first column) and convert to date
    date_column_index = 0
    value_column_index = 2  # Third column contains the values

    df.iloc[:, date_column_index] = pd.to_datetime(df.iloc[:, date_column_index], format='%d.%m.%y').dt.date

    # Create a DataFrame with only the Date and Value columns
    df = df.iloc[:, [date_column_index, value_column_index]]
    df.columns = ['Date', 'Value']

    # Set 'Date' as the index for easier manipulation
    df.set_index('Date', inplace=True)

    # Create a complete range of daily dates between the min and max Date
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D').date

    # Reindex the DataFrame to include all dates in the range
    df = df.reindex(full_range)

    # Print missing dates
    missing_dates = df[df['Value'].isna()].index
    for missing_date in missing_dates:
        print(f"Missing date found: {missing_date}")

    # Reset the index and fill missing 'Value' entries with NaN
    df.reset_index(inplace=True)
    df.columns = ['Date', 'Value']

    return df




### Process all data as Downloaded from LUIS (Environmental data tool city of Graz)
http://app.luis.steiermark.at/luft2/suche.php?station1=&station2=&komponente1=&station3=&station4=&komponente2=&von_tag=1&von_monat=1&von_jahr=2009&mittelwert=21&bis_tag=31&bis_monat=12&bis_jahr=2013

In [ ]:
strRelPathToDownloads = "../datasets/raw_2009-2013/downloads"
strRelPathOutput = "../datasets/raw_2009-2013/processed"

liFiles = os.listdir(strRelPathToDownloads)

for strFile in liFiles:
    strFilePath = os.path.join(strRelPathToDownloads,strFile)
    sheet_name = None
    filled_df = fill_missing_values(strFilePath, sheet_name)
    filled_df.to_excel(os.path.join(strRelPathOutput,"PRO_"+strFile), index=False)
    print(f"Filled DataFrame saved to {strFilePath}")


### Copy manually all columns (station by station) and store it into data_per_station_2009-2013

### Define dictionary containing all station dataframes and read data

In [173]:
dict_df_stations_2009_2013 = {'d':pd.DataFrame(),'w':pd.DataFrame(),'s':pd.DataFrame(),'n':pd.DataFrame(),'e':pd.DataFrame()}

for strKey in dict_df_stations_2009_2013:
    dict_df_stations_2009_2013[strKey] = pd.read_excel("../datasets/data_per_station_2009-2013/"+strKey+".xls",index_col=0)  # Set the fifth row as the header
dict_df_stations_2009_2013['d']

,temp,pm10,rh
2009-01-02,-5.822104,76.490000,86.730840
2009-01-03,-4.172874,53.750000,90.160835
2009-01-04,-4.812709,62.070007,85.871666
2009-01-05,-6.369397,89.330025,82.762090
2009-01-06,-3.787598,112.439964,79.990000
...,...,...,...
2013-12-27,7.975574,19.289993,87.726460
2013-12-28,6.086780,18.250000,90.061180
2013-12-29,2.726640,22.150010,94.296364
2013-12-30,3.986715,18.229994,90.518394


### Define functions needed to add temporal features

In [174]:


# the below is the same, but takes a string:
at_holidays = holidays.country_holidays(subdiv='6',country="AT")
hr_holidays = holidays.country_holidays(country="HR")
def check_holiday(date,country_hol)->int:
    if date in country_hol:
        return 1
    else:
        return 0
        
def check_weekend(date)->int:
    if date.weekday() > 4: #goes from 0..6
        return 1
    else:
        return 0

def get_day_of_year(date):
    return date.timetuple().tm_yday


### Add temporal features

In [175]:
for df in dict_df_stations_2009_2013:  
    dict_df_stations_2009_2013[df]["dayOfYear"] =  [get_day_of_year(date) for date in dict_df_stations_2009_2013[df].index.to_pydatetime()]
    dict_df_stations_2009_2013[df]["holiday"] =  [check_holiday(date,at_holidays) for date in dict_df_stations_2009_2013[df].index]
    dict_df_stations_2009_2013[df]["dayAfterHoliday"] =  [check_holiday(date-datetime.timedelta(days=1),at_holidays) for date in dict_df_stations_2009_2013[df].index]
    dict_df_stations_2009_2013[df]["dayBeforeHoliday"] =  [check_holiday(date+datetime.timedelta(days=1),at_holidays) for date in dict_df_stations_2009_2013[df].index]
    dict_df_stations_2009_2013[df]["weekend"] =  [check_weekend(date) for date in dict_df_stations_2009_2013[df].index]


### Add pm10 lag values

In [176]:
for df in dict_df_stations_2009_2013:  
    dict_df_stations_2009_2013[df]["pm10Lag"] = dict_df_stations_2009_2013[df]["pm10"].shift(1) 

### Drop data annually 1st to 3rd January (New year)

In [177]:
li_drop_dates_new_year = ['01-01', '01-02','01-03']

for df in dict_df_stations_2009_2013:  
    # uncomment to see all values to be dropped
    #print("New year values")
    #display(dict_df_stations[df][dict_df_stations[df].index.strftime('%m-%d').isin(li_drop_dates_new_year)])

    index_to_drop_graz = dict_df_stations_2009_2013[df][dict_df_stations_2009_2013[df].index.strftime('%m-%d').isin(li_drop_dates_new_year)].index
    dict_df_stations_2009_2013[df].drop(index_to_drop_graz,inplace=True)


### Find NaN values

In [178]:
def calc_output_nans(df):
    total_nan = 0
    for elem in list(df.keys()):
        length = len(df.loc[df[elem].isnull(),])
        total_nan += length
        if length > 0:
            print("Feature: "+ elem, "count: ",length)
    print("Total NaN: ",total_nan)


calc_output_nans(dict_df_stations_2009_2013["d"])
#calc_output_nans(dict_df_stations_2009_2013["e"])
#calc_output_nans(dict_df_stations_2009_2013["n"])
#calc_output_nans(dict_df_stations_2009_2013["s"])
#calc_output_nans(dict_df_stations_2009_2013["w"])


Feature: temp count:  18
Feature: pm10 count:  26
Feature: rh count:  1
Feature: pm10Lag count:  26
Total NaN:  71


### Find rows in which NaN values occur - important to know how many instance contain NaN

In [179]:
def checkNaN(dict_df,detailed=True):
    li_stations = ["d","n","e","s","w"]
    total_nas = 0
    for station in dict_df:
        df = dict_df[station]
        #print(dict_df_stations_2009_2013[station])
        total_nas += len(df[df.isna().any(axis=1)])
        #print("No. of NAs: ",len(df[df.isna().any(axis=1)]))
    
        if detailed:
            display(df[df.isna().any(axis=1)])
    print("No. of total NAs: ",total_nas)

checkNaN(dict_df_stations_2009_2013,False)



No. of total NAs:  2056


### Impute Values

In [181]:

# Station east no temp, hum, press
# imputed from nearby station south
dict_df_stations_2009_2013["e"][["temp"]] = dict_df_stations_2009_2013["s"][["temp"]]
dict_df_stations_2009_2013["e"][["rh"]] = dict_df_stations_2009_2013["s"][["rh"]]


### Check NaN again

In [182]:
checkNaN(dict_df_stations_2009_2013,False)


No. of total NAs:  291


### 291 NaN instances (rows in which a NaN value occurs) 

- 1825 instances per station --> 5 *1825 = 9125 total instances

- 291 / 9125 = 0.0318 --> 3.1% of NaNs

### Store values

In [184]:
for name,df in dict_df_stations_2009_2013.items():
    filepath = Path('../datasets/data_per_station_2009-2013/'+name+'.csv')
    dict_df_stations_2009_2013[name].to_csv(filepath,sep=',')

### Read data from 2014-2021, concat with 2009-2013, while only using similiar features

In [187]:
dict_df_stations_2014_2021 = {'d':pd.DataFrame(),'w':pd.DataFrame(),'s':pd.DataFrame(),'n':pd.DataFrame(),'e':pd.DataFrame()}

dict_df_stations_total = {'d':pd.DataFrame(),'w':pd.DataFrame(),'s':pd.DataFrame(),'n':pd.DataFrame(),'e':pd.DataFrame()}

station_id = 0
for strKey in dict_df_stations_2014_2021:
    dict_df_stations_2014_2021[strKey] = pd.read_csv("../datasets/data_per_station/"+strKey+".csv",index_col=0,parse_dates=True)  # Set the fifth row as the header
    dict_df_stations_2009_2013[strKey] = pd.read_csv("../datasets/data_per_station_2009-2013/"+strKey+".csv",index_col=0,parse_dates=True)  # Set the fifth row as the header

    # get common features
    station_id = dict_df_stations_2014_2021[strKey]["id"]
    liFeatures_2014_2021 = dict_df_stations_2014_2021[strKey].columns.to_list()
    liFeatures_2009_2013 = dict_df_stations_2009_2013[strKey].columns.to_list()
    liCommonFeatures = list(set(liFeatures_2014_2021) & set(liFeatures_2009_2013))

    # concat to one large dataframe
    dict_df_stations_total[strKey] = dfConCat = pd.concat([dict_df_stations_2009_2013[strKey][liCommonFeatures],dict_df_stations_2014_2021[strKey][liCommonFeatures]])

    


,rh,dayOfYear,dayBeforeHoliday,pm10,weekend,holiday,pm10Lag,temp,dayAfterHoliday
2009-01-04,85.871666,4,0,62.070007,1,0,53.750000,-4.812709,0
2009-01-05,82.762090,5,1,89.330025,0,0,62.070007,-6.369397,0
2009-01-06,79.990000,6,0,112.439964,0,1,89.330025,-3.787598,0
2009-01-07,87.087910,7,0,80.979990,0,0,112.439964,-4.472021,1
2009-01-08,85.682060,8,0,86.800026,0,0,80.979990,-4.773334,0
...,...,...,...,...,...,...,...,...,...
2021-11-21,95.958333,325,0,27.362500,1,0,28.837500,2.754167,0
2021-11-22,95.083333,326,0,31.566667,0,0,27.362500,4.316667,0
2021-11-23,91.000000,327,0,33.000000,0,0,31.566667,3.791667,0
2021-11-24,87.500000,328,0,38.900000,0,0,33.000000,0.654167,0


### Check the length of the new dataset

In [195]:
cnt = 0
for station in dict_df_stations_total:
    cnt += len(dict_df_stations_total[station]) 
    print("Station: ",station,len(dict_df_stations_total[station]))
print("Total rows: ",cnt)

Station:  d 4668
Station:  w 4668
Station:  s 4668
Station:  n 4668
Station:  e 4668
Total rows:  23340


### Store combined dataset

In [196]:
for name,df in dict_df_stations_total.items():
    filepath = Path('../datasets/data_per_station_2009-2022/'+name+'.csv')
    dict_df_stations_total[name].to_csv(filepath,sep=',')